# How to use easy campaign from python


## RHFE

### SETUP

In [1]:
# prepare Setup
from openff.units import unit
from rdkit import Chem
from openfe import SmallMoleculeComponent, SolventComponent

suppl = Chem.SDMolSupplier("molecules/rhfe/benzenes_RHFE.sdf")
ligands = [SmallMoleculeComponent.from_rdkit(m) for m in suppl]
solvent = SolventComponent(ion_concentration=0.15*unit.molar)

# Prepare setup Planning
from openfe import setup
from openfe.protocols.openmm_rbfe import RelativeLigandProtocol

atom_mapper = setup.LomapAtomMapper()
atom_mapping_scorer = setup.lomap_scorers.default_lomap_score
ligand_network_planner = setup.MinimalSpanningNetworkPlanner(mappers=[atom_mapper], mapping_scorer=atom_mapping_scorer)

rbfe_protocol = RelativeLigandProtocol(settings=RelativeLigandProtocol._default_settings())
alchemical_network_planner = setup.RHFEAlchemicalNetworkPlanner(name="easy_rhfe", ligand_network_planner=ligand_network_planner, protocol=rbfe_protocol)

LICENSE: Could not open license file "oe_license.txt" in local directory
LICENSE: N.B. OE_LICENSE environment variable is not set
LICENSE: N.B. OE_DIR environment variable is not set
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!


## Plan Calculations

In [2]:
alchemical_network = alchemical_network_planner(ligands=ligands, solvent=solvent)
alchemical_network

<AlchemicalNetwork-3b01cda7aabaa5c3de173d532eba88ca>

In [5]:
vars(alchemical_network).edges

AttributeError: 'dict' object has no attribute '_edges'

In [13]:
print("Transformations to calculate:", len(alchemical_network.edges))
for edge in alchemical_network.edges:
    print("\t",edge.name)

Transformations to calculate: 30
	 easy_rhfe_lig_3_solvent_lig_14_solvent
	 easy_rhfe_lig_7_solvent_lig_13_solvent
	 easy_rhfe_lig_7_vacuum_lig_13_vacuum
	 easy_rhfe_lig_12_vacuum_lig_15_vacuum
	 easy_rhfe_lig_8_solvent_lig_9_solvent
	 easy_rhfe_lig_5_vacuum_lig_10_vacuum
	 easy_rhfe_lig_13_solvent_lig_15_solvent
	 easy_rhfe_lig_2_solvent_lig_3_solvent
	 easy_rhfe_lig_10_vacuum_lig_15_vacuum
	 easy_rhfe_lig_13_vacuum_lig_14_vacuum
	 easy_rhfe_lig_8_vacuum_lig_9_vacuum
	 easy_rhfe_lig_1_vacuum_lig_9_vacuum
	 easy_rhfe_lig_15_vacuum_lig_16_vacuum
	 easy_rhfe_lig_2_vacuum_lig_3_vacuum
	 easy_rhfe_lig_6_solvent_lig_9_solvent
	 easy_rhfe_lig_6_vacuum_lig_9_vacuum
	 easy_rhfe_lig_15_solvent_lig_16_solvent
	 easy_rhfe_lig_12_solvent_lig_15_solvent
	 easy_rhfe_lig_4_vacuum_lig_7_vacuum
	 easy_rhfe_lig_4_solvent_lig_7_solvent
	 easy_rhfe_lig_3_vacuum_lig_14_vacuum
	 easy_rhfe_lig_5_solvent_lig_10_solvent
	 easy_rhfe_lig_11_solvent_lig_13_solvent
	 easy_rhfe_lig_1_solvent_lig_9_solvent
	 easy_rh

### Calculations
in this stage the planned transfromations are going to be calculated.

**WARNING:** These scripts shall serve as examples. The RHFE calculations can take very long and you should have HPC environment where you can parallelize the execution. - here in the future our Exorcist Project will help you

In [14]:
# Output:
output_dir = "output"   

In [18]:
from gufe.protocols.protocoldag import execute_DAG

transformation_results = []
for transformation in alchemical_network.edges:
    
    print("Running the simulations: ", transformation.name)
    transformation_task_dag = transformation.create()
    transformation_task_result = execute_DAG(transformation_task_dag)
    transformation_result = transformation.protocol.gather([transformation_task_result])
    transformation_results.append(transformation_result)
print("Done")

INFO:	creating hybrid system


Running the simulations:  easy_rhfe_lig_3_solvent_lig_14_solvent


INFO:	Requested to generate parameters for residue <Residue 0 (UNK) of chain 0>
INFO:	Generating a residue template for [H][O][C@@]1([H])[C]([C]([H])([H])[H])=[C]([H])[C]([H])([H])[C@]([H])([C](=[C]([H])[H])[C]([H])([H])[H])[C]1([H])[H] using openff-2.0.0.offxml


KeyboardInterrupt: 

/home/bries/programs/miniconda3/envs/openfe/bin/wrapped_progs/antechamber: Fatal Error!
Cannot open file (sqm.out) with mode (r).
No such file or directory


## gather results:

In [ ]:
for transformation_result in transformation_results:
    if transformation_result.ok():
        estimate = transformation_result.get_estimate()
        uncertainty = transformation_result.get_uncertainty()
    else:
        estimate = uncertainty = None
    print("Estimate: ", transformation_result.name, estimate, uncertainty)